In [6]:
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize as wt
from nltk import PorterStemmer  
import re
from nltk import FreqDist
from nltk.probability import LidstoneProbDist, WittenBellProbDist
import sys
from IPython.display import display
#sys.stdout = stdout
#reload(sys)
#sys.setdefaultencoding("utf-8")
stopWord = set(sw.words("english"));
import numpy as np
import pandas as pds
import math
import os
from __future__ import print_function

In [7]:
#variables

In [8]:
files = ["ray_913742943690653739_cf.txt", "ray_925987435331224445_cf.txt", 
        "ray_925987537478928034_pf.txt"]
lam = 0.3
topNRDoc = 10;

In [9]:
#function to clean document 

In [10]:
short = {"n't":"not"}
def getwords(doc):
    w = doc.split(' ')
    nw = w[:]
    for i in range (len(w)):
        col = w[i].find("\'")
        l = len(w[i]);
        if l - col == 2:
            ss = nw[i][col:];
            if ss in short:
                ss = short[ss]
            nw.append(nw[i][col-1:])
            nw[i] = nw[i][:col-1]
    sent = " ".join(nw);
    return sent;

In [11]:
dots = ['.', ',', '!', '', "+", "#", "(", ")", ":", "'s"]
slangs = {"n't":"not", "r": "are", "u": "you"}
def goclean(doc):
    doc = doc.lower()
    #\W+
    tokens = wt(doc)#getwords(doc)#re.findall('\w+', doc)
    filterWord = [];
    
    for w in tokens:
        if w not in dots:
            if w in slangs:
                w = slangs[w];
            filterWord.append(w)
    sents = " ".join(filterWord)
    filterWord = re.findall('\w+', sents)
    ps = PorterStemmer()
    stemed = []
    for w in filterWord:
        stemed.append(ps.stem(w)) 
    return stemed

In [12]:
#function to calculate probability

In [13]:
def getprobability(cleanW):
    myarray = np.asarray(cleanW)
    ue, ce = np.unique(myarray, return_counts=True)
    tc = len(cleanW);
    n = float(tc)
    
    p = [];
    for i in range(len(ce)): 
        p.append(ce[i]/n)
    
    mydict = dict(zip(ue, p))
    return mydict

In [14]:
#smoothing function

In [15]:
def smooth(probd, probc, lam):
    dict2 = dict(probc)
    
    for key, items in probc.iteritems():
        if key in probd:
            val = (1 - lam)*probd[key] + lam*probc[key];
            #pdf[name][key] = val;
            dict2[key] = val
        
        else:
            val = lam*(probc[key]);
            #pdf[name][key] = val;
            dict2[key] = val
    return dict2;

In [16]:
def getpratio(probd, probc, lam):
    dict2 = dict(probc)
    
    for key, items in probc.iteritems():
        if key in probd:
            val = ((1 - lam)*probd[key])/(lam*probc[key]) + 1;
            #pdf[name][key] = val;
            dict2[key] = val
        
        else:
            val = lam*(probc[key]);
            #pdf[name][key] = val;
            dict2[key] = val
    return dict2;

In [17]:
#KL divergence

In [18]:
def KLdivergence(q, df):
    x = [0]*td;
    for word, prob in q.iteritems():
        if word not in pdf.index:
            continue;
        for i in range(td):
            name = 'probd' + str(i)
            x[i] = x[i] - prob*(math.log(pdf[name][word]))
    return x;       

In [19]:
#working on collection

In [20]:
doc = ""
def getdoc(files, doc):
    for f in files:
        doc += open(f , 'r').read()
    return doc

doc += getdoc(files, doc);
doc = doc.decode('utf-8');
cw = goclean(doc)

In [21]:
probc = getprobability(cw)
dict2 = dict(probc)
pdf = pds.DataFrame(dict2.values(), index=dict2.keys(), columns=["probc"])
pdf["probc"]["1"]
tc = len(probc)

In [22]:
#Working on reviews

In [23]:
reviews = doc.split("\n")
td = len(reviews)
for i in range(len(reviews)):
    name = "probd" + str(i);
    clrev = goclean(reviews[i])
    prob = getprobability(clrev)
    ps = getpratio(prob, probc, lam)
    pdf[name] = ps.values()

In [24]:
#query modeling

In [25]:
#q = (reviews[16] + '.')[:-1]
q = raw_input()
cq = goclean(q)
qprob = getprobability(cq)
div = KLdivergence(qprob, pdf)
index = div.index(min(div))
reviews[index]

voice


u'Very clear voice and content is good.'

In [26]:
#top N Relevent reviews

In [30]:
tlist = div[:];
x = []#*topNRDoc
for i in range(topNRDoc):
    least = min(tlist)
    if least == 0:
        break
    index = tlist.index(least);
    x.append(index)
    tlist[index] = float("inf")
    
rdl = len(x)
if rdl == 0:
    print("No relevent document found")
    
else:
    prt = ""
    for i in range(topNRDoc):
        display(str(i+1) + ') ' + reviews[x[i]])

u'1) Very clear voice and content is good.'

u'2) Voice was clear and concise from a tone perspective.'

u'3) Good voice inflection. Good example of recommendation, relating to ML.'

u'4) Use of examples and cases, length okay, clear voice, closing'

u'5) Good effort Lisa! Good voice projection. You will get better with more practices.'

u'6) Very enjoyable use of voice and tone and engaging delivery. Hit all of the key messages and core points and value prop from the script.'

u"7) Pace and varied voice inflection helps keep the attention of the audience. Clearly didn't feel rushed, which made it easy to follow. Good use of example use cases."

u"8) You're a credible and engaging speaker great voice dynamics, facial expressions, cadence. Your delivery helps make the content seem accessible, simple, understandable. Content was complete. Thank you for your second effort on this!!"

u'9) Enjoyed your passion in your voice for AWS Ml. Very simple, clean to understand. Liked how you referred to ML in their daily life like "cashier-less grocery shopping". Liked that you emphasized our desire to give everyone a choice.'

u"10) You're a credible and engaging speaker excellent pacing, voice dynamics, etc. Love that you included stat re: number of Amazonians working on ML (100K), in addition to tenure (20 yrs). Good frame up of the three layers, and explanations of each."